In [57]:
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matsim
import random

In [58]:
T=pd.read_csv("travelDiaryDeciles_npdc.csv", ";",encoding='utf8', dtype=str) # matrix trips

In [36]:
T.columns

Index(['DECH', 'PECH', 'COEQ', 'DP2', 'D4A', 'D4B', 'D3', 'D3X', 'D3Y', 'D2AA',
       'D8A', 'D8B', 'D7', 'D7X', 'D7Y', 'D5AA', 'MODP', 'NDEP', 'SUP_D3',
       'SUP_D7', 'CODEDEPT_D3', 'CODEREG_D3', 'CODEDEPT_D7', 'CODEREG_D7',
       'MTDQ', 'PTDQ', 'P4', 'P2', 'P5', 'P10', 'M5', 'DOIB', 'DIST', 'D8C',
       'D9', 'M6A', 'M6B', 'M6C', 'M6D', 'CODEDEPT_PP2', 'CODEREG_PP2',
       'CODEDEPT_DP2', 'CODEREG_DP2', 'Q115', 'Q215', 'Q315', 'Q3_Q1', 'D115',
       'D215', 'D315', 'D415', 'D615', 'D715', 'D815', 'D915', 'ECH_x'],
      dtype='object')

In [37]:
T['DIST'] = T['DIST'].astype(float)
T['D8C'] = T['D8C'].astype(float)
T['COEQ'] = T['COEQ'].astype(float)

In [61]:
# Perform stochastic rounding for the population (and scale weights)
def recovery(df, id_data, weight_data, old_id_data, random=1234):
    df_cp = df.copy()
    random = np.random.RandomState(random)
    df_rounding = df_cp[[id_data, weight_data]].drop_duplicates(id_data)
    df_rounding["multiplicator"] = np.floor(df_rounding[weight_data])##Recuperation des parties entières
    df_rounding["multiplicator"] += random.random_sample(len(df_rounding)) <= (df_rounding[weight_data] - df_rounding[weight_data])###Arrondissement
    df_rounding["multiplicator"] = df_rounding["multiplicator"].astype(np.int)

    df_multiplicators = df_rounding["multiplicator"].values
    df_cp = df_cp.iloc[np.repeat(np.arange(len(df_cp)), df_multiplicators)]##Dupliquer les lignes autant de fois qu'il y a poids de personnes pour la ligne

    # Create new person IDs
    df_cp[old_id_data] = df_cp[id_data]
    df_cp[id_data] = np.arange(len(df_cp))

    del df_cp[weight_data]
    
    return df_cp

In [62]:
Tr = recovery(T.copy(), "DECH", "COEQ", "rec_trip_id", 1234)
Tr.reset_index(drop=True)

/tmp/ipykernel_30679/2436349370.py:8: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  df_rounding["multiplicator"] = df_rounding["multiplicator"].astype(np.int)


,DECH,PECH,DP2,D4A,D4B,D3,D3X,D3Y,D2AA,D8A,...,D115,D215,D315,D415,D615,D715,D815,D915,ECH_x,rec_trip_id
0,0,01200400010100401,62119,09,45,62119,674661.80202,7048040.28762,01,09,...,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004,0120040001010040101
1,1,01200400010100401,62119,09,45,62119,674661.80202,7048040.28762,01,09,...,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004,0120040001010040101
2,2,01200400010100401,62119,09,45,62119,674661.80202,7048040.28762,01,09,...,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004,0120040001010040101
3,3,01200400010100401,62119,09,45,62119,674661.80202,7048040.28762,01,09,...,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004,0120040001010040101
4,4,01200400010100401,62119,09,45,62119,674661.80202,7048040.28762,01,09,...,8858,10837,12729.62962962963,14623,19225.416666666668,21900.666666666668,25877.33333333333,33081.333333333336,012004000101004,0120040001010040101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12886929,12886929,302014456009094701,80021,16,30,59606,736767.05632,7029166.66083,02,18,...,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947,30201445600909470102
12886930,12886930,302014456009094701,80021,16,30,59606,736767.05632,7029166.66083,02,18,...,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947,30201445600909470102
12886931,12886931,302014456009094701,80021,16,30,59606,736767.05632,7029166.66083,02,18,...,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947,30201445600909470102
12886932,12886932,302014456009094701,80021,16,30,59606,736767.05632,7029166.66083,02,18,...,8911.724137931034,11167.560975609756,13250.5,15356.8,20084,22956,27043.33333333333,34545,3020144560090947,30201445600909470102


In [64]:
Tr.to_csv("tripNpdc_recovered.csv", sep=";", encoding="utf8", index=False)

In [59]:
Tr = pd.read_csv("tripNpdc_recovered.csv", ";",encoding='utf8', dtype=str) # matrix trips

In [60]:
Tcar=Tr[(Tr["MODP"]=="21") | (Tr["MODP"]=="22") | (Tr["MODP"]=="61") | (Tr["MODP"]=="81") | (Tr["MODP"]=="93") | 
       (Tr["MODP"]=="99")]
TcarHdF = Tcar[(Tcar["CODEREG_D3"] == "32") & (Tcar["CODEREG_D7"] == "32")]

In [84]:
# Flows between region
Rc = Tr.copy().groupby(['CODEREG_D3','CODEREG_D7'])['DECH'].count().reset_index()
# Flows between department
Dc = Tr.copy().groupby(['CODEDEPT_D3','CODEDEPT_D7'])['DECH'].count().reset_index()

c1 = Dc["CODEDEPT_D3"] == "59"
c2 = Dc["CODEDEPT_D3"] == "62"
c3 = Dc["CODEDEPT_D3"] == "02"
c4 = Dc["CODEDEPT_D3"] == "60"
c5 = Dc["CODEDEPT_D3"] == "80"
c6 = (c1) | (c2) | (c3) | (c4) | (c5)

d1 = Dc["CODEDEPT_D7"] == "59"
d2 = Dc["CODEDEPT_D7"] == "62"
d3 = Dc["CODEDEPT_D7"] == "02"
d4 = Dc["CODEDEPT_D7"] == "60"
d5 = Dc["CODEDEPT_D7"] == "80"
d6 = (d1) | (d2) | (d3) | (d4) | (d5)

# Flux departements

In [514]:
# car flows in HdF
TcarHdF = Tcar[(Tcar["CODEREG_D3"] == "32") & (Tcar["CODEREG_D7"] == "32")]
# ndep
df = TcarHdF.copy().groupby(['CODEDEPT_D3','CODEDEPT_D7'])['DECH'].count().unstack(fill_value=0).stack().reset_index()
df=df.rename(columns={0:"DECH"})
df.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash/input/ndep.csv", sep="\t", encoding="utf8", index=False, header=None)
# ndep per 10000
df10000 = df.copy()
df10000["DECH"] = (df10000["DECH"]/10000).astype(int)
df10000.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash/input/ndep10000.csv", sep="\t", encoding="utf8", index=False, header=None)

In [509]:
# keep departments
DcarHdF = TcarHdF.copy().groupby(['CODEDEPT_D3','CODEDEPT_D7'])['DECH'].count().reset_index()
d3 = DcarHdF["CODEDEPT_D3"]
d7 = DcarHdF["CODEDEPT_D7"]
d = pd.concat([d3,d7], axis=0, ignore_index=True)
d = d.drop_duplicates(keep='first')
# zones
d.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash/input/zones.csv", sep="\t", encoding="utf8", index=False, header=None)

In [517]:
#ndep = df10000
ndep = df
ndepa = ndep[["CODEDEPT_D3","DECH"]]
ndepa=ndepa.copy().groupby(['CODEDEPT_D3'])['DECH'].sum().reset_index()
ndepb = ndep[["CODEDEPT_D7","DECH"]]
ndepb=ndepb.rename(columns={"CODEDEPT_D7":"CODEDEPT_D3", "DECH":"DECHD7"})
ndepb = ndepb.copy().groupby(['CODEDEPT_D3'])['DECHD7'].sum().reset_index()
ndepab = pd.merge(ndepa.copy(), ndepb.copy(), how='left',on ="CODEDEPT_D3")
ndepab = ndepab.fillna(0)
ndepab['sum'] = round((ndepab['DECH'] + ndepab['DECHD7'])/10000)
ndepab = ndepab[["CODEDEPT_D3","sum"]]
ndepab["sum"][ndepab["sum"] == 0.0] = 1.0
ndepab.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash/input/bornemax.csv", sep="\t", encoding="utf8", index=False, header=None)

In [519]:
# nombre de bornes n
n=((ndepab["sum"].sum())/10).astype(int)
bornes=[i for i in range(n)]
bornes=pd.DataFrame(bornes)
bornes.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash/input/bornes.csv", sep="\t", encoding="utf8", index=False, header=None)

In [521]:
# distance entre zones
df2 = TcarHdF.copy().groupby(['CODEDEPT_D3','CODEDEPT_D7'])['DIST'].mean().unstack(fill_value=0).stack().reset_index()
df2=df2.rename(columns={0:"DIST"})
df2["DIST"] = df2["DIST"].astype(int)
df2.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash/input/distance.csv", sep="\t", encoding="utf8", index=False, header=None)

# Flux communauté de communes

In [54]:
comm=pd.read_csv("/home/ndiop/Documents/phd/scripts/demandgeneration2/codes/input_hdf/communesXY.csv", ",",encoding='utf8', dtype=str) # matrix trips

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [10]:
TcarHdF.columns

Index(['DECH', 'PECH', 'DP2', 'D4A', 'D4B', 'D3', 'D3X', 'D3Y', 'D2AA', 'D8A',
       'D8B', 'D7', 'D7X', 'D7Y', 'D5AA', 'MODP', 'NDEP', 'SUP_D3', 'SUP_D7',
       'CODEDEPT_D3', 'CODEREG_D3', 'CODEDEPT_D7', 'CODEREG_D7', 'MTDQ',
       'PTDQ', 'P4', 'P2', 'P5', 'P10', 'M5', 'DOIB', 'DIST', 'D8C', 'D9',
       'M6A', 'M6B', 'M6C', 'M6D', 'CODEDEPT_PP2', 'CODEREG_PP2',
       'CODEDEPT_DP2', 'CODEREG_DP2', 'Q115', 'Q215', 'Q315', 'Q3_Q1', 'D115',
       'D215', 'D315', 'D415', 'D615', 'D715', 'D815', 'D915', 'ECH_x',
       'rec_trip_id'],
      dtype='object')

In [11]:
'''Cette fonction permet de remplacer les codes INSEE par les coordonnées (X et Y), les départements et les communes
    Elle renvoie en sortie les nouvelle données avec les coordonnées X et Y, les codes départementaux et communaux
    Elle prend comme entrées: data, data_xy et output_file
        data est les données avec les code INSEE (déplacement.txt),
        data_xy les données de correspondance code INSEE et (X, Y),
        output_file est le chemin complet du fichier de sortie
'''

# Remplacement des codes INSEE par leur correspondance
def mapping_codeINSEE_xy(data, data_xy):
    #select columns
    data_xy.columns =["ID_GEOFLA","INSEE_COM","INSEE_CODE","NOM_COMM","STATUT","X_CHF_LIEU","Y_CHF_LIEU","X_CENTROID",
                         "Y_CENTROID","Z_MOYEN","SUPERFICIE","POPULATION","CODE_CANT","CODE_ARR","CODE_DEPT","NOM_DEPT",
                         "CODE_REG","NOM_REGION","NUM_D30","NOM_D30","Xcord","Ycord"]
# mapping    
    # mapping code INSEE avec commune (nécssaire pour l'anayse des flux : pour les variables PP2, DP2, D3 et D7)
    map_com = dict(zip(data_xy.INSEE_CODE, data_xy.NUM_D30))

    
# création de colonnes  
    # créer des colonnes pour CODE_COM
    data["CODECOM_D3"]=data["D3"]
    data["CODECOM_D7"]=data["D7"]
    data["CODECOM_DP2"]=data["DP2"]    
        
# remplacement    
    # remplacer les codes INSEE par les communes
    data[["CODECOM_D3"]] = data[["CODECOM_D3"]].applymap(map_com.get)
    data[["CODECOM_D7"]] = data[["CODECOM_D7"]].applymap(map_com.get)
    data[["CODECOM_DP2"]] = data[["CODECOM_DP2"]].applymap(map_com.get)
    
    # suppression des données dont les coordonnées X,Y ne figurent pas dans la base ComunesXY.csv ==> travelDiary_hdf
    data = data.dropna(subset=["CODECOM_D3","CODECOM_D7"])
    
    #data[cols].to_csv(output_file,index=None, sep=";")
    
    return data


In [12]:
TcarHdF1 = mapping_codeINSEE_xy(TcarHdF.copy(),comm)

In [47]:
# for qgis analysis
origin = TcarHdF1.CODECOM_D3.value_counts()
destination = TcarHdF1.CODECOM_D7.value_counts()
od = pd.concat([origin,destination], axis=1, ignore_index=True)
od['sum'] = od[0] + od[1]
od['sum%'] = od["sum"]*100/od["sum"].sum()
df2.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/od.csv", sep="\t", encoding="utf8", index=False)

In [48]:
# ndep
df = TcarHdF1.copy().groupby(['CODECOM_D3','CODECOM_D7'])['DECH'].count().unstack(fill_value=0).stack().reset_index()
df=df.rename(columns={0:"DECH"})
df.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/ndep.csv", sep="\t", encoding="utf8", index=False, header=None)
# ndep per 10000
df10000 = df.copy()
df10000["DECH"] = (df10000["DECH"]/10000).astype(int)
df10000.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/ndep10000.csv", sep="\t", encoding="utf8", index=False, header=None)

In [55]:
# keep communes
DcarHdF = df.copy().groupby(['CODECOM_D3','CODECOM_D7'])['DECH'].count().reset_index()
d3 = DcarHdF["CODECOM_D3"]
d7 = DcarHdF["CODECOM_D7"]
d = pd.concat([d3,d7], axis=0, ignore_index=True)
d = d.drop_duplicates(keep='first')
# zones
d.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/zones.csv", sep="\t", encoding="utf8", index=False, header=None)

In [64]:
#ndep = df10000
ndep = df
ndepa = ndep[["CODECOM_D3","DECH"]]
ndepa=ndepa.copy().groupby(['CODECOM_D3'])['DECH'].sum().reset_index()
ndepb = ndep[["CODECOM_D7","DECH"]]
ndepb=ndepb.rename(columns={"CODECOM_D7":"CODECOM_D3", "DECH":"DECHD7"})
ndepb = ndepb.copy().groupby(['CODECOM_D3'])['DECHD7'].sum().reset_index()
ndepab = pd.merge(ndepa.copy(), ndepb.copy(), how='left',on ="CODECOM_D3")
ndepab = ndepab.fillna(0)
ndepab['sum'] = round((ndepab['DECH'] + ndepab['DECHD7'])/10000)
ndepab = ndepab[["CODECOM_D3","sum"]]
ndepab["sum"][ndepab["sum"] == 0.0] = 1.0
ndepab.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/bornemax.csv", sep="\t", encoding="utf8", index=False, header=None)

In [66]:
# nombre de bornes n
n=((ndepab["sum"].sum())/10).astype(int)
bornes=[i for i in range(n)]
bornes=pd.DataFrame(bornes)
bornes.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/bornes.csv", sep="\t", encoding="utf8", index=False, header=None)

In [13]:
# distance entre zones
TcarHdF1['DIST'] = TcarHdF1['DIST'].astype(float)
df2 = TcarHdF1.copy().groupby(['CODECOM_D3','CODECOM_D7'])['DIST'].mean().unstack(fill_value=0).stack().reset_index()
df2=df2.rename(columns={0:"DIST"})
df2["DIST"] = df2["DIST"].astype(int)
df2.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/input/distance.csv", sep="\t", encoding="utf8", index=False, header=None)

# Flux communes

In [61]:
# car flows in HdF
TcarHdF = Tcar[(Tcar["CODEREG_D3"] == "32") & (Tcar["CODEREG_D7"] == "32")]

In [62]:
len(TcarHdF["D3"].unique())

1786

In [63]:
len(TcarHdF["D7"].unique())

1791

In [64]:
TcarHdF2 = TcarHdF.copy()
d3 = TcarHdF2["D3"]
d7 = TcarHdF2["D7"]
d3a = d3.unique()
d7a = d7.unique()
d3a=list(d3a)
d7a=list(d7a)
list_difference = []
for item in d3a:
  if item not in d7a:
    list_difference.append(item)
print(list_difference)

['60537', '60509', '80622', '80670', '80733', '02720', '02158', '02109', '02833', '02013', '80122', '80328', '02709', '80220', '02359', '02006', '02647', '02760', '02494', '02079', '02264', '02418', '02278', '02481', '02100', '02237', '02758', '60139', '60647', '60533', '02810', '80167', '80810', '80714', '80444', '80166', '80316', '80351', '80092', '80585', '80802', '80741', '80417', '80302']


In [65]:
TcarHdF3 = TcarHdF2.copy()[~TcarHdF2.copy()['D3'].isin(list_difference)]

In [66]:
len(TcarHdF3["D3"].unique())

1742

In [67]:
TcarHdF4 = TcarHdF3.copy()
d3 = TcarHdF4["D3"]
d7 = TcarHdF4["D7"]
d3a = d3.unique()
d7a = d7.unique()
d3a=list(d3a)
d7a=list(d7a)
list_difference2 = []
for item in d7a:
  if item not in d3a:
    list_difference2.append(item)
print(list_difference2)

['80373', '80129', '80211', '80057', '02231', '59277', '80801', '02067', '62664', '80115', '02785', '02199', '80359', '02319', '02658', '02659', '02757', '02313', '02469', '02668', '02040', '02286', '02357', '02400', '02419', '02775', '02738', '02180', '02091', '02294', '60141', '60074', '60175', '60338', '60600', '60665', '60025', '60145', '80008', '80440', '80765', '80526', '80155', '80550', '80020', '80752', '80164', '80521', '80070']


In [68]:
TcarHdF5 = TcarHdF4.copy()[~TcarHdF4.copy()['D7'].isin(list_difference2)]

In [69]:
len(TcarHdF5["D7"].unique())

1742

In [16]:
TcarHdF5.to_csv("tripNpdcCar_recovered.csv", sep=";", encoding="utf8", index=False)

In [3]:
TcarHdF5 = pd.read_csv("tripNpdcCar_recovered.csv", ";",encoding='utf8', dtype=str) # matrix trips

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [70]:
# ndep
dfi = TcarHdF5.copy().groupby(['D3','D7'])['DECH'].count().unstack(fill_value=0).stack().reset_index()
dfi=dfi.rename(columns={0:"DECH"})
dfi.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash2/input/ndep.csv", sep="\t", encoding="utf8", index=False, header=None)
# ndep per 10000
dfi10000 = dfi.copy()
dfi10000["DECH"] = (dfi10000["DECH"]/10000).astype(int)
dfi10000.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash2/input/ndep10000.csv", sep="\t", encoding="utf8", index=False, header=None)

In [74]:
# keep communes
DcarHdF = dfi.copy().groupby(['D3','D7'])['DECH'].count().reset_index()
d3 = DcarHdF["D3"]
d7 = DcarHdF["D7"]
d = pd.concat([d3,d7], axis=0, ignore_index=True)
d = d.drop_duplicates(keep='first')
# zones
d.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash2/input/zones.csv", sep="\t", encoding="utf8", index=False, header=None)

In [76]:
# nombre de bornes maximales par communes
#ndep = df10000
ndep = dfi
ndepa = ndep[["D3","DECH"]]
ndepa=ndepa.copy().groupby(['D3'])['DECH'].sum().reset_index()
ndepb = ndep[["D7","DECH"]]
ndepb=ndepb.rename(columns={"D7":"D3", "DECH":"DECHD7"})
ndepb = ndepb.copy().groupby(['D3'])['DECHD7'].sum().reset_index()
ndepab = pd.merge(ndepa.copy(), ndepb.copy(), how='left',on ="D3")
ndepab = ndepab.fillna(0)
ndepab['sum'] = round((ndepab['DECH'] + ndepab['DECHD7'])/8000) # to defined for realistic number of stations:8000
ndepab = ndepab[["D3","sum"]]
ndepab["sum"][ndepab["sum"] == 0.0] = 1.0
ndepab.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash2/input/bornemax.csv", sep="\t", encoding="utf8", index=False, header=None)

In [21]:
ndepab["sum"].sum()

3190.0

In [22]:
# nombre de bornes n
n=((ndepab["sum"].sum())/10).astype(int)
bornes=[i for i in range(n)]
bornes=pd.DataFrame(bornes)
bornes.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash2/input/bornes.csv", sep="\t", encoding="utf8", index=False, header=None)

In [4]:
# distance entre zones
TcarHdF5['DIST'] = TcarHdF5['DIST'].astype(float)
df2 = TcarHdF5.copy().groupby(['D3','D7'])['DIST'].mean().unstack(fill_value=0).stack().reset_index()
df2=df2.rename(columns={0:"DIST"})
df2["DIST"] = df2["DIST"].astype(int)
df2.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash2/input/distance.csv", sep="\t", encoding="utf8", index=False, header=None)

In [7]:
# Other method
def SummarizeTrip(df):
    df2=pd.DataFrame()
    df2["D3"]=""
    df2["D7"]=""
    df2["NBR"]=""
    df2["DURMOY"]=""
    df2["DISTMOY"]=""
    n = len(df)
    k=0
    while(n>0):
        a = (df["D3"]==df.loc[0,"D3"]) & (df["D7"]==df.loc[0,"D7"])
        dfi=df[a]     #recuperer les lignes contenant le D3 et le D7 courant (i)
        Ind = dfi.index.values
        dfi.reset_index(drop=True, inplace=True)
        df2.loc[k,"NBR"]= len(dfi)
        df2.loc[k,"DURMOY"]= (dfi["D8C"]).astype(float).mean()  
        df2.loc[k,"DISTMOY"]= (dfi["DIST"]).astype(float).mean()  
        df2.loc[k,"D3"]= dfi.loc[0,"D3"] 
        df2.loc[k,"D7"]= dfi.loc[0,"D7"] 
        df.drop(Ind, inplace=True)
        df.reset_index(drop=True,inplace=True)
        n = len(df)
        print(k,n)
        k=k+1
    return df2
Mcar = SummarizeTrip(Tcar.copy())

# Evaluation charging optimisation

In [140]:
B=pd.read_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/bornesactivee.csv", ";",encoding='utf8') # matrix trips

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [142]:
B1=B[0:171]
B2=B[171:343]

In [145]:
B1.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/b1.csv", sep=";", encoding="utf8", index=False, header=True)
B2.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/b2.csv", sep=";", encoding="utf8", index=False, header=False)

In [214]:
B1=pd.read_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/b1.csv", ";",encoding='utf8') # matrix trips
B2=pd.read_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/b2.csv", ";",encoding='utf8') # matrix trips

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [216]:
B2.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'],1,inplace=True)

/tmp/ipykernel_8162/963268992.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  B2.drop(['Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19'],1,inplace=True)


In [220]:
B2=B2.rename(columns={"B2":"B1"})

In [221]:
bb = pd.merge(B1.copy(), B2.copy(), how='left',on ="B1")

In [222]:
bb.drop("B1",1,inplace=True)

/tmp/ipykernel_8162/191660896.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  bb.drop("B1",1,inplace=True)


In [329]:
bb1 = bb.copy() 
bb1.loc['total'] = bb1.select_dtypes(pd.np.number).sum()

/tmp/ipykernel_8162/3964731535.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  bb1.loc['total'] = bb1.select_dtypes(pd.np.number).sum()


In [330]:
Br = bb1.iloc[[-1]]

In [331]:
Bp = Br.T

In [332]:
Bp = Bp.rename_axis('D30b', axis=1)

In [335]:
Bp.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/bp1.csv", sep=";", encoding="utf8", index=True, header=True)

In [336]:
Bp=pd.read_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/bp1.csv", ";",encoding='utf8') # matrix trips

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [338]:
Bp=Bp.rename(columns={"Unnamed: 0":"D30b"})

In [340]:
D30HdF=pd.read_csv("/home/ndiop/Documents/phd/visualisation/EMD_HdF_2018/tmp/d30HDF.csv", ",",encoding='utf8', dtype="str") 

/home/ndiop/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  exec(code_obj, self.user_global_ns, self.user_ns)


In [346]:
Bp1 = pd.concat([Bp.copy(),D30HdF.copy()], axis=1, ignore_index=False)

In [349]:
Bp1.to_csv("/home/ndiop/Documents/phd/scripts/ampl/dept/out/lastfiles/validationbash3/outputAMPL/out1000bornes/bornesD30HDF.csv", sep=";", encoding="utf8", index=True, header=True)

In [350]:
Bp1

,D30b,total,NUM_D30,NOM_D30
0,1,0.0,001,Sud de l'Aisne
1,2,0.0,002,Saint-Quentinois
2,3,0.0,003,Laonnois
3,4,0.0,004,Soissonnais
4,5,0.0,005,Thiérache
5,6,0.0,006,Chaunois
6,7,0.0,007,Cambrésis
7,8,107.0,008,Valenciennois
8,9,0.0,009,Douaisis
9,10,92.0,010,Sambre-Avesnois
